In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)


In [2]:
data_raw = pd.read_csv('../../DATA/FINAL/transformed_set_enriched.csv')

In [3]:
KEYS = [
    'entity_id',
    'financial_report_id',
    'financial_statement_id',
    'entity_ico',
    'entity_name']

DIMENSIONS = [
    'year',
    # 'month',
    # 'financial_report_order_number',
    'period_length',
    # 'consolidated',
    # 'approved_date',
    'type',
    'financial_report_template_id',
    # 'establishment_date',
    # 'cancellation_date',
    # 'sknace_code',
    # 'entity_state',
    'legal_form_code',
    'org_size_code',
    'ownership_category_code',
    'entity_consolidated',
    # 'sknace_division_name',
    'sknace_division',
    'sknace_division_normalized',
    # 'sknace_subcategory',
    'sal_class']

RAW_VALUES = [ 
    'total_assets',
    'non_current_assets',
    'non_current_intangible_assets',
    'non_current_tangible_assets',
    'non_current_financial_assets',
    'current_assets',
    'inventories',
    'non_current_receivables',
    'current_receivables',
    'financial_assets',
    'current_financial_assets',
    'cash_and_bank_balances',
    'total_equity_and_liabilities',
    'equity',
    'share_capital',
    'share_premium',
    'net_profit_of_previous_years',
    'net_profit_after_tax',
    'liabilities',
    'non_current_liabilities',
    'long_term_provisions',
    'long_term_bank_loans',
    'current_liabilities',
    'short_term_provisions',
    'current_bank_loans',
    'short_term_financial_assistance',
    'operating_revenues',
    'sales_from_the_merchandise',
    'sales_from_the_own_products_and_services',
    'sales_from_the_other',
    'other_revenues_from_operating_activities',
    'operating_costs',
    'costs_of_merchandise_sold',
    'consumed_materials_energy_non_inventory_supplies',
    'services',
    'personnel_costs',
    'taxes_and_fees',
    'depreciation',
    'remaining_cost_of_sold_long_term_assets_and_materials',
    'adjustments_to_receivables',
    'other_costs_of_operating_activities',
    'operating_result',
    'value_added',
    'revenues_from_financial_activities',
    'cost_of_financial_activities',
    'result_from_fincancial_activities',
    'result_before_tax',
    'income_tax',
    'result_after_tax']


RATIOS = ['L1',
       'L2', 'L3', 'NCL_CF', 'CASH_TA', 'INV_COGS', 'CC_SAL', 'TL_COST',
       'CL_COST', 'SAL_TA', 'EQ_TA', 'EQ_TL', 'CL_CC', 'LTC_NCA', 'TZ',
       'CF_TL', 'CL_TA', 'NCL_TA', 'EBT_REV', 'ROS', 'EAT_REV', 'ROA', 'ROE',
       'CF_TA', 'CF_SAL', 'LABOR_EAT', 'LABOR_REV']

PRIMARY_VALUES = [
    'CA', 'CASH',
       'CL', 'EQ', 'NCL', 'TA', 'TL', 'CC', 'INV', 'CF_NETTO', 'CF_SELFFIN',
       'REV', 'VA', 'OE', 'EAT', 'EBIT', 'SAL', 'COST', 'INT'
]



DISTRESS_SIGNALS = [
    'DPHZ_vat_registration_cancelled',
    'DPHZ_vat_registration_cancelled_1y_off',
    'DPHZ_vat_registration_cancelled_2y_off',
    'DPHZ_vat_registration_cancelled_3y_off',
    'RU_event',
    'RU_event_1y_off',
    'RU_event_2y_off',
    'RU_event_3y_off',
    'cancellation_year',
    'RUZ_cancelled',
    'RUZ_cancelled_1y_off',
    'RUZ_cancelled_2y_off',
    'RUZ_cancelled_3y_off']

In [4]:

entity_details_table = pd.read_csv('../../DATA/TRANSFORM/entity_details/financial_statements_register_entity_details.csv')
entity_details_table = entity_details_table[['id', 'entity_name']].rename(columns={'id': 'entity_id'})
entity_details_table

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (13,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,entity_id,entity_name
0,4.0,"NESS Slovensko, a.s."
1,63.0,"EKOCHEM, a.s. v likvidácií"
2,74.0,"ZTS Sabinov, a. s."
3,184.0,HPK engineering a.s.
4,185.0,"CHEMINVEST, a.s."
...,...,...
410569,2096976.0,"MERIN SERVICES, s. r. o."
410570,2096979.0,ELDA-BAU s. r. o.
410571,2096986.0,Derma Medical s.r.o.
410572,2096989.0,Cmetal s. r. o.


In [5]:
NOT_RELEVANT = ['T', 'U', 'O']
RELEVANT = ['H', 'C', 'M', 'F', 'G']

def sknace_grouping(x):
    if x in ['T', 'U', 'O']:
        return 'non-relevant'
    if x not in ['H', 'C', 'M', 'F', 'G']:
        return 'other'
    return x

data_raw['sknace_division_normalized'] = data_raw['sknace_division'].apply(sknace_grouping)

In [6]:
data = data_raw.drop(columns=RAW_VALUES)
display(data.shape[0])
data = data.drop(columns = 'entity_name').merge(entity_details_table.drop_duplicates('entity_id'), on ='entity_id', how = 'left')
data_raw = None
display(data.shape[0])

955333

955333

In [7]:
def parse_name(x):
    if 'v likvidacii' in x or 'v likvidácii' in x:
        return 'likvidacia'
    
    if 'v konkurze' in x:
        return 'konkurz'
    
    if 'v restrukturalizacii' in x or 'v reštrukturalizácii' in x:
        return 'restrukturalizacia'

    return 'ok'

data['parsed_state'] = data['entity_name'].apply(parse_name)

In [8]:
# pomer VI a zavazkov je mensi ako 8, t.j EQ_TL < 0.08
# 2016 : 0.04, 2017 : 0.06, 2018 : 0.08, 
# EQ < 0
# L3 < 1


def process_issues(row):
    EQ_TL = row.EQ_TL
    EQ = row.EQ
    L3 = row.L3
    EAT = row.EAT
    y = row.year

    if y <= 2016: threat_criterion = 0.04
    elif y == 2017: threat_criterion = 0.06
    else: threat_criterion = 0.08

    if EQ_TL < threat_criterion and EQ < 0 and L3 < 1 and EAT < 0:
        return 1
    return 0
    

In [9]:
ruz_indirect_list = []

for _, row in data.iterrows():
    criteria_value = process_issues(row)
    ruz_indirect_list.append([row['entity_id'], row['year'], row['period_length'], criteria_value])

ruz_indirect_df = pd.DataFrame(ruz_indirect_list, columns=['entity_id', 'year', 'period_length', 'RUZ_indirect_criteria'])


In [10]:
ruz_indirect_df['period_length_check'] = ruz_indirect_df['period_length'].apply(lambda x: 1 if x == 12 else 0)
ruz_indirect_df

ruz_indirect_df = (ruz_indirect_df
                   .sort_values(['period_length_check', 'RUZ_indirect_criteria'], ascending=[False, False])
                   .groupby(['entity_id', 'year'])
                   .first()
                   .reset_index())[['entity_id', 'year', 'RUZ_indirect_criteria']]

In [11]:
def process_entity_issues_2y_off(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'])

def process_entity_issues_3y_off(row, issue_type):
        return max(row[issue_type+'_1y_off'], row[issue_type+'_2y_off'], row[issue_type+'_3y_off'])


entity_issues_indicators_1y_off = ruz_indirect_df.copy()
entity_issues_indicators_2y_off = ruz_indirect_df.copy()
entity_issues_indicators_3y_off = ruz_indirect_df.copy()


entity_issues_indicators_1y_off['year'] = entity_issues_indicators_1y_off['year'] - 1
entity_issues_indicators_1y_off.rename(columns={'RUZ_indirect_criteria':'RUZ_indirect_criteria_1y_off'}, inplace=True)

entity_issues_indicators_2y_off['year'] = entity_issues_indicators_2y_off['year'] - 2
entity_issues_indicators_2y_off.rename(columns={'RUZ_indirect_criteria':'RUZ_indirect_criteria_2y_off'}, inplace=True)

entity_issues_indicators_3y_off['year'] = entity_issues_indicators_3y_off['year'] - 3
entity_issues_indicators_3y_off.rename(columns={'RUZ_indirect_criteria':'RUZ_indirect_criteria_3y_off'}, inplace=True)


entity_issues_indicators = (ruz_indirect_df
                            .merge(
                                entity_issues_indicators_1y_off,
                                on = ['entity_id', 'year'],
                                how = 'outer'
                            )                           
                            .merge(
                                entity_issues_indicators_2y_off,
                                on = ['entity_id', 'year'],
                                how = 'outer'
                            )                                                    
                            .merge(
                                entity_issues_indicators_3y_off,
                                on = ['entity_id', 'year'],
                                how = 'outer'
                            ))

for col in ['RUZ_indirect_criteria',
       'RUZ_indirect_criteria_1y_off', 'RUZ_indirect_criteria_2y_off',
       'RUZ_indirect_criteria_3y_off']:
       entity_issues_indicators[col] = entity_issues_indicators[col].fillna(0)

entity_issues_indicators['RUZ_indirect_criteria_2y_off'] = entity_issues_indicators.apply(lambda x: process_entity_issues_2y_off(x, 'RUZ_indirect_criteria'), axis = 1)
entity_issues_indicators['RUZ_indirect_criteria_3y_off'] = entity_issues_indicators.apply(lambda x: process_entity_issues_3y_off(x, 'RUZ_indirect_criteria'), axis = 1)

entity_issues_indicators


,entity_id,year,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off
0,4.0,2014.0,0.0,0.0,0.0,0.0
1,4.0,2015.0,0.0,0.0,0.0,0.0
2,4.0,2016.0,0.0,0.0,0.0,0.0
3,4.0,2017.0,0.0,0.0,0.0,1.0
4,4.0,2018.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...
1420963,1611042.0,2011.0,0.0,0.0,0.0,0.0
1420964,1611565.0,2011.0,0.0,0.0,0.0,0.0
1420965,1634614.0,2012.0,0.0,0.0,0.0,0.0
1420966,1635085.0,2012.0,0.0,0.0,0.0,1.0


In [12]:
data = data.merge(entity_issues_indicators, on = ['entity_id', 'year'], how = 'left')

In [13]:
DISTRESS_SIGNALS = [
    'DPHZ_vat_registration_cancelled',
    'DPHZ_vat_registration_cancelled_1y_off',
    'DPHZ_vat_registration_cancelled_2y_off',
    'DPHZ_vat_registration_cancelled_3y_off',
    'RU_event',
    'RU_event_1y_off',
    'RU_event_2y_off',
    'RU_event_3y_off',
    'cancellation_year',
    'RUZ_cancelled',
    'RUZ_cancelled_1y_off',
    'RUZ_cancelled_2y_off',
    'RUZ_cancelled_3y_off',
    'RUZ_indirect_criteria',
    'RUZ_indirect_criteria_1y_off',
    'RUZ_indirect_criteria_2y_off',
    'RUZ_indirect_criteria_3y_off']


In [14]:
data.query('sal_class in ["2. Malý", "3. Stredný"]')['cancellation_year'].value_counts()

2017.0    488
2021.0    442
2020.0    422
2022.0    345
2016.0    324
2018.0    303
2019.0    247
2015.0     89
2014.0      1
Name: cancellation_year, dtype: int64

In [15]:
def get_sample_sizes(data, row_label, column_label):
    samples_df = data.groupby([row_label, column_label])['financial_statement_id'].nunique().to_frame('count').reset_index()
    wide_df = pd.pivot_table(samples_df, index = row_label, columns=column_label, fill_value=0)
    return wide_df

In [16]:
def combined_criteria(row, years_offset):
    return max(row[f'RU_event_{int(years_offset)}y_off'], row[f'RUZ_cancelled_{int(years_offset)}y_off'], row[f'RUZ_indirect_criteria_{int(years_offset)}y_off'])

data['combined_issue_criteria_1y_off'] = data.apply(lambda x: combined_criteria(x, 1), axis = 1)
data['combined_issue_criteria_2y_off'] = data.apply(lambda x: combined_criteria(x, 2), axis = 1)
data['combined_issue_criteria_3y_off'] = data.apply(lambda x: combined_criteria(x, 3), axis = 1)

data

,entity_id,year,month,financial_report_id,financial_report_template_id,CA,CASH,CL,EQ,NCL,TA,TL,CC,INV,CF_NETTO,CF_SELFFIN,REV,VA,OE,EAT,EBIT,SAL,COST,INT,LABOR,L1,L2,L3,NCL_CF,CASH_TA,INV_COGS,CC_SAL,TL_COST,CL_COST,SAL_TA,EQ_TA,EQ_TL,CL_CC,LTC_NCA,TZ,CF_TL,CL_TA,NCL_TA,EBT_REV,ROS,EAT_REV,ROA,ROE,CF_TA,CF_SAL,LABOR_EAT,LABOR_REV,financial_statement_id,financial_report_order_number,entity_ico,period_length,consolidated,approved_date,type,establishment_date,cancellation_date,sknace_code,entity_state,legal_form_code,org_size_code,ownership_category_code,entity_consolidated,sknace_division_name,sknace_division,sknace_subcategory,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off,cancellation_year,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_cancelled_3y_off,sal_class,establishment_year,sknace_division_normalized,entity_name,parsed_state,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off,combined_issue_criteria_1y_off,combined_issue_criteria_2y_off,combined_issue_criteria_3y_off
0,4.0,2014.0,1.0,4644773,699.0,8787618.0,5238983.0,4658457.0,2112122.0,3232.0,11200988.0,8657266.0,2578606.0,16429.0,1171078.0,1063283.0,19361305.0,3910128.0,408169.0,1007897.0,1115692.0,18580340.0,18172171.0,4446.0,3384185.0,1.124618,1.678150,1.681677,0.002760,0.467725,NaN,0.138781,0.476402,0.256351,1.658813,0.188566,0.196123,1.806580,2.532115,0.702983,0.135271,0.415897,0.000289,0.057625,0.054245,0.052057,0.089983,0.477196,0.104551,0.063028,3.357669,0.174791,2382229.0,1.0,603783.0,12.0,NaN,2015-07-07,Riadna,1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,1991,other,"NESS Slovensko, a.s.",ok,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,2015.0,1.0,5382540,699.0,20276962.0,5687157.0,6153737.0,13229589.0,2890.0,20862929.0,7184264.0,14037315.0,6792.0,3940396.0,3245262.0,29573853.0,4276968.0,6057021.0,6536244.0,7231378.0,17608945.0,11551924.0,2910.0,3581817.0,0.924179,3.205284,3.206387,0.000733,0.272596,NaN,0.797170,0.621911,0.532702,0.844030,0.634119,0.648069,0.438384,24.336039,-3.184198,0.548476,0.294960,0.000139,0.244519,0.371189,0.221014,0.313295,0.494063,0.188871,0.223772,0.547993,0.121114,2968194.0,1.0,603783.0,12.0,NaN,2016-09-28,Riadna,1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,1991,other,"NESS Slovensko, a.s.",ok,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,2016.0,1.0,5665728,699.0,5991927.0,1828391.0,4859422.0,1178778.0,3568.0,6518310.0,5120402.0,3840942.0,5894.0,247140.0,11702.0,12914606.0,3565411.0,313595.0,-44282.0,191156.0,12878215.0,12564620.0,38229.0,3476051.0,0.376257,1.166668,1.167881,0.014437,0.280501,NaN,0.298251,0.407525,0.386754,1.975698,0.180841,0.187132,1.265164,2.735191,-2.244983,0.048266,0.745503,0.000547,0.014802,-0.003439,-0.003429,-0.006793,-0.037566,0.037915,0.019191,-78.498058,0.269157,3194585.0,3.0,603783.0,12.0,NaN,2017-06-12,Riadna,1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,1991,other,"NESS Slovensko, a.s.",ok,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,2017.0,1.0,6245237,699.0,3016170.0,1224056.0,2649399.0,268147.0,3224.0,3442598.0,2902134.0,1355249.0,1954.0,-1327325.0,-1211994.0,7725983.0,1952011.0,-1442061.0,-1270731.0,-1386062.0,7651564.0,9093625.0,9037.0,3304021.0,0.462013,0.973543,0.974281,-0.002429,0.355562,NaN,0.177121,0.319139,0.291347,2.222613,0.077891,0.084581,1.954917,1.221500,-0.241746,-0.457362,0.769593,0.000937,-0.179403,-0.166075,-0.164475,-0.369120,-4.738934,-0.385559,-0.173471,-2.600095,0.427651,3662172.0,2.0,603783.0,12.0,NaN,2018-06-26,Riadna,1991-01-17,NaN,62090.0,NaN,121.

In [17]:
temp = data.query('period_length == 12 and REV > 0 and TA > 0')[['entity_id', 'year', 'REV', 'cancellation_year', 'RU_event_3y_off', 'RUZ_cancelled_3y_off', 'sknace_division', 'financial_statement_id']]

In [18]:
def get_weighted_sample_sizes(data, row_label, column_label, weight):
    samples_df = data.groupby([row_label, column_label])[weight].sum().to_frame('count').reset_index()
    wide_df = pd.pivot_table(samples_df, index = row_label, columns=column_label, fill_value=0)
    return wide_df

In [19]:
get_weighted_sample_sizes(temp, 'sknace_division', 'year', 'REV')

count                                            \
year                   2014.0        2015.0        2016.0        2017.0   
sknace_division                                                           
A                1.845145e+09  1.861335e+09  1.866057e+09  1.909732e+09   
B                2.639898e+08  3.009151e+08  2.507712e+08  3.102062e+08   
C                3.383035e+10  3.635382e+10  3.569986e+10  3.555930e+10   
D                2.313414e+09  2.559443e+09  2.595885e+09  2.505948e+09   
E                6.291571e+08  6.515005e+08  6.567736e+08  7.003198e+08   
F                6.057562e+09  7.260728e+09  6.204776e+09  6.576768e+09   
G                3.965855e+10  4.146950e+10  3.982411e+10  4.102945e+10   
H                6.325830e+09  6.675447e+09  6.683663e+09  6.961080e+09   
I                1.194480e+09  1.294759e+09  1.430401e+09  1.329613e+09   
J                4.709275e+09  5.263808e+09  4.438437e+09  4.265324e+09   
K                1.327743e+09  1.268702e+09  1.197676e+09  1.194630e+09   
L                3.592499e+09  4.191923e+09  3.496930e+09  3.727360e+09   
M                7.355686e+09  8.528643e+09  7.350072e+09  6.755715e+09   
N                5.897303e+09  4.845289e+09  4.589283e+09  4.746717e+09   
O                1.570006e+07  1.816355e+07  1.961015e+07  1.935285e+07   
P                1.662146e+08  2.032159e+08  1.646313e+08  1.874884e+08   
Q                1.379591e+09  1.447532e+09  1.544136e+09  1.616782e+09   
R                1.687449e+09  1.972224e+09  2.247861e+09  2.342629e+09   
S                1.968979e+08  2.073349e+08  2.275591e+08  2.453531e+08   

                                                                      
year                  2018.0        2019.0       2020.0       2021.0  
sknace_division                                                       
A                 2005903025  2.008761e+09   2196973734   2188476780  
B                  329235052  3.294965e+08    214863565    319995485  
C                37824178506  3.673066e+10  33987814370  36757186392  
D                 2858521973  3.006665e+09   2584085285   2511019345  
E                  745254502  7.710469e+08    743173363    948339567  
F                 7043552849  6.987609e+09   6201035235   6329303480  
G                42663808566  4.261747e+10  40045424643  42327017290  
H                 7222108697  7.186020e+09   6451083634   6959046218  
I                 1398932857  1.528410e+09   1058426946   1052802032  
J                 4596605120  4.933766e+09   4367479005   4526487892  
K                 1178932085  1.354164e+09   1221844996    816886507  
L                 3650757481  3.649856e+09   3000938569   3075590319  
M                 6608924389  6.589614e+09   6245539468   6151988883  
N                 4635926463  4.558547e+09   3604937454   3825662410  
O                   24172140  2.581302e+07     24819238     15851414  
P                  183872723  1.954846e+08    174441066    197570485  
Q                 1720984776  1.872790e+09   1884772983   2250731745  
R                 2572294909  2.630828e+09   1408230274   1651282887  
S                  273796161  2.581937e+08    231960894    236796140

In [20]:
def get_liquidation_3y_off(row):
    if row.RUZ_cancelled_3y_off == 1 and row.parsed_state != 'ok':
        return 1
    return 0

def get_liquidation_2y_off(row):
    if row.RUZ_cancelled_2y_off == 1 and row.parsed_state != 'ok':
        return 1
    return 0

def get_liquidation_1y_off(row):
    if row.RUZ_cancelled_1y_off == 1 and row.parsed_state != 'ok':
        return 1
    return 0


data['RUZ_liquidation_3y_off'] = data.apply(get_liquidation_3y_off, axis = 1)
data['RUZ_liquidation_2y_off'] = data.apply(get_liquidation_2y_off, axis = 1)
data['RUZ_liquidation_1y_off'] = data.apply(get_liquidation_1y_off, axis = 1)



data.query('RUZ_cancelled_3y_off == 1 and parsed_state != "ok"')['RUZ_liquidation_3y_off'].value_counts()

1    15098
Name: RUZ_liquidation_3y_off, dtype: int64

In [21]:
plausible_entity_ids = pd.DataFrame()

for current_year in 2015, 2016, 2017, 2018:
    previous_year = current_year - 1
    two_consequent_years = set(temp.query('year == @current_year')['entity_id']).intersection(temp.query('year == @previous_year')['entity_id'])

    one_off = current_year + 1
    two_off = current_year + 2
    three_off = current_year + 3

    all_three = (set(temp.query('year == @one_off')['entity_id'])
                 .intersection(temp.query('year == @two_off')['entity_id'])
                 .intersection(temp.query('year == @three_off')['entity_id']))
    
    two_and_cancelled = (set(temp.query('year == @one_off')['entity_id'])
                         .intersection(temp.query('year == @two_off')['entity_id'])
                         .intersection(temp.query('cancellation_year == @three_off')['entity_id']))
    
    one_and_cancelled = (set(temp.query('year == @one_off')['entity_id'])
                         .intersection(temp.query('cancellation_year == @two_off')['entity_id']))
    
    next_cancelled = set(temp.query('cancellation_year == @one_off')['entity_id'])

    ru_event_pass = set(temp.query('year == @current_year and RU_event_3y_off == 1 or RUZ_cancelled_3y_off == 1')['entity_id'])

    future_set = (all_three
                  .union(two_and_cancelled)
                  .union(one_and_cancelled)
                  .union(next_cancelled)
                  .union(ru_event_pass))
    
    total = two_consequent_years.intersection(future_set)

    yearly_plausible_entity_ids = pd.DataFrame(total, columns = ['entity_id'])
    yearly_plausible_entity_ids['year'] = current_year
    
    plausible_entity_ids = pd.concat([plausible_entity_ids, yearly_plausible_entity_ids], axis = 0)

plausible_entity_ids['plausible_sample'] = 1

subset_data = data.merge(plausible_entity_ids, on = ['entity_id', 'year'], how = 'left')
subset_data['plausible_sample'] = subset_data['plausible_sample'].fillna(0)

subset_data

,entity_id,year,month,financial_report_id,financial_report_template_id,CA,CASH,CL,EQ,NCL,TA,TL,CC,INV,CF_NETTO,CF_SELFFIN,REV,VA,OE,EAT,EBIT,SAL,COST,INT,LABOR,L1,L2,L3,NCL_CF,CASH_TA,INV_COGS,CC_SAL,TL_COST,CL_COST,SAL_TA,EQ_TA,EQ_TL,CL_CC,LTC_NCA,TZ,CF_TL,CL_TA,NCL_TA,EBT_REV,ROS,EAT_REV,ROA,ROE,CF_TA,CF_SAL,LABOR_EAT,LABOR_REV,financial_statement_id,financial_report_order_number,entity_ico,period_length,consolidated,approved_date,type,establishment_date,cancellation_date,sknace_code,entity_state,legal_form_code,org_size_code,ownership_category_code,entity_consolidated,sknace_division_name,sknace_division,sknace_subcategory,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off,cancellation_year,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_cancelled_3y_off,sal_class,establishment_year,sknace_division_normalized,entity_name,parsed_state,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off,combined_issue_criteria_1y_off,combined_issue_criteria_2y_off,combined_issue_criteria_3y_off,RUZ_liquidation_3y_off,RUZ_liquidation_2y_off,RUZ_liquidation_1y_off,plausible_sample
0,4.0,2014.0,1.0,4644773,699.0,8787618.0,5238983.0,4658457.0,2112122.0,3232.0,11200988.0,8657266.0,2578606.0,16429.0,1171078.0,1063283.0,19361305.0,3910128.0,408169.0,1007897.0,1115692.0,18580340.0,18172171.0,4446.0,3384185.0,1.124618,1.678150,1.681677,0.002760,0.467725,NaN,0.138781,0.476402,0.256351,1.658813,0.188566,0.196123,1.806580,2.532115,0.702983,0.135271,0.415897,0.000289,0.057625,0.054245,0.052057,0.089983,0.477196,0.104551,0.063028,3.357669,0.174791,2382229.0,1.0,603783.0,12.0,NaN,2015-07-07,Riadna,1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,1991,other,"NESS Slovensko, a.s.",ok,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0
1,4.0,2015.0,1.0,5382540,699.0,20276962.0,5687157.0,6153737.0,13229589.0,2890.0,20862929.0,7184264.0,14037315.0,6792.0,3940396.0,3245262.0,29573853.0,4276968.0,6057021.0,6536244.0,7231378.0,17608945.0,11551924.0,2910.0,3581817.0,0.924179,3.205284,3.206387,0.000733,0.272596,NaN,0.797170,0.621911,0.532702,0.844030,0.634119,0.648069,0.438384,24.336039,-3.184198,0.548476,0.294960,0.000139,0.244519,0.371189,0.221014,0.313295,0.494063,0.188871,0.223772,0.547993,0.121114,2968194.0,1.0,603783.0,12.0,NaN,2016-09-28,Riadna,1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,1991,other,"NESS Slovensko, a.s.",ok,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0
2,4.0,2016.0,1.0,5665728,699.0,5991927.0,1828391.0,4859422.0,1178778.0,3568.0,6518310.0,5120402.0,3840942.0,5894.0,247140.0,11702.0,12914606.0,3565411.0,313595.0,-44282.0,191156.0,12878215.0,12564620.0,38229.0,3476051.0,0.376257,1.166668,1.167881,0.014437,0.280501,NaN,0.298251,0.407525,0.386754,1.975698,0.180841,0.187132,1.265164,2.735191,-2.244983,0.048266,0.745503,0.000547,0.014802,-0.003439,-0.003429,-0.006793,-0.037566,0.037915,0.019191,-78.498058,0.269157,3194585.0,3.0,603783.0,12.0,NaN,2017-06-12,Riadna,1991-01-17,NaN,62090.0,NaN,121.0,11.0,7.0,True,"Počítačové programovanie, poradenstvo a súvisi...",J,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,0,0,0,3. Stredný,1991,other,"NESS Slovensko, a.s.",ok,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1.0
3,4.0,2017.0,1.0,6245237,699.0,3016170.0,1224056.0,2649399.0,268147.0,3224.0,3442598.0,2902134.0,1355249.0,1954.0,-1327325.0,-1211994.0,7725983.0,1952011.0,-1442061.0,-1270731.0,-1386062.0,7651564.0,9093625.0,9037.0,3304021.0,0.462013,0.973543,0.974281,-0.002429,0.355562,NaN,0.177121,0.319139,0.291347,2.222613,0.077891,0.084581,1.954917,1.221500,-0.241746,-0.457362,0.769593,0.000937,-0.179403,-0.166075,-0.164475,-0.369120,-4.738934,-0.3

In [22]:
subset_data = subset_data.query('plausible_sample == 1 and sal_class in ["2. Malý", "3. Stredný"] and period_length == 12 and RUZ_indirect_criteria == 0 and RUZ_cancelled == 0 and RU_event == 0 and (RUZ_cancelled_3y_off == 0 or parsed_state != "ok")')

In [23]:
master_sample = subset_data.query('RU_event_3y_off == 1').sample(frac=1, random_state=7).groupby('entity_id').first().reset_index()
master_sample_set = set(master_sample['entity_id'])

ruz_cancelled_entity = subset_data.query('RUZ_cancelled_3y_off == 1 and entity_id not in @master_sample_set').sample(frac=1, random_state=7).groupby('entity_id').first().reset_index()

master_sample = pd.concat([master_sample, ruz_cancelled_entity], axis = 0)
master_sample_set = set(master_sample['entity_id'])

ruz_indirect_criteria = subset_data.query('RUZ_indirect_criteria_3y_off == 1 and entity_id not in @master_sample_set').sample(frac=1, random_state=7).groupby('entity_id').first().reset_index()

master_sample = pd.concat([master_sample, ruz_indirect_criteria], axis = 0)
master_sample_set = set(master_sample['entity_id'])

rest = subset_data.query('entity_id not in @master_sample_set').sample(frac=1, random_state=7).groupby('entity_id').first().reset_index()

master_sample = pd.concat([master_sample, rest], axis = 0)

display(master_sample['entity_id'].value_counts())

27971.0      1
486300.0     1
485687.0     1
485759.0     1
485802.0     1
            ..
180469.0     1
180453.0     1
180437.0     1
180423.0     1
1611042.0    1
Name: entity_id, Length: 8988, dtype: int64

In [24]:
for dim in ['RU_event_3y_off', 'RUZ_cancelled_3y_off', 'RUZ_indirect_criteria_3y_off', 'combined_issue_criteria_3y_off']:
    print(dim)
    display(master_sample[dim].value_counts(normalize=True))

RU_event_3y_off


0.0    0.97441
1.0    0.02559
Name: RU_event_3y_off, dtype: float64

RUZ_cancelled_3y_off


0    0.988763
1    0.011237
Name: RUZ_cancelled_3y_off, dtype: float64

RUZ_indirect_criteria_3y_off


0.0    0.926235
1.0    0.073765
Name: RUZ_indirect_criteria_3y_off, dtype: float64

combined_issue_criteria_3y_off


0.0    0.902982
1.0    0.097018
Name: combined_issue_criteria_3y_off, dtype: float64

In [25]:
from sklearn.ensemble import IsolationForest

from sklearn.preprocessing import PowerTransformer
import sys

def detect_outliers_isolation_forest(df):
    filled_data = fill_missing_values_mean(df)

    isolation_forest = IsolationForest(contamination=0.02, random_state=42) 
    outliers = isolation_forest.fit_predict(filled_data)
    
    outlier_mask = pd.Series(outliers == -1, index=filled_data.index)

    return outlier_mask


def yeo_johnson_transform(data):
    pt = PowerTransformer(method='yeo-johnson')
    columns = data.columns
    index = data.index
    data = pd.DataFrame(pt.fit_transform(data), columns=columns, index=index)
    return data


def fill_missing_values_mean(data):
    data_filled = data.fillna(data.mean())
    data_filled_fallback = data_filled.fillna(0)
    return data_filled_fallback


def mahalanobis_distance(data):
    filled_data = fill_missing_values_mean(data)

    transformed_data = yeo_johnson_transform(filled_data)
    normalized_data = (transformed_data - transformed_data.mean()) / transformed_data.std()
    
    mean_vector = normalized_data.mean()
    covariance_matrix = normalized_data.cov()

    if np.linalg.cond(covariance_matrix) > 1 / sys.float_info.epsilon:
        regularization = 1e-6
        covariance_matrix += np.eye(covariance_matrix.shape[0]) * regularization

    try:
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)

        distances = []
        for index, row in normalized_data.iterrows():
            diff = row - mean_vector
            distance = np.sqrt(np.dot(np.dot(diff.T, inv_covariance_matrix), diff))
            distances.append(distance)

        return pd.Series(distances, index=data.index)

    except np.linalg.LinAlgError:
        print("Error inverting covariance matrix. Data might be singular.")
        return pd.Series([], index=data.index)

def flag_outliers_mahalanobis(data, threshold=3):
    distances = mahalanobis_distance(data)
    chi2_threshold = threshold ** 2

    try:
        cutoff = np.percentile(distances, 100 * (1 - np.exp(-chi2_threshold / 2)))
        outliers = distances > cutoff
        return outliers

    except Exception as e:
        print(f"Error calculating cutoff: {e}")
        return pd.Series([], index=data.index)



In [26]:

outlier_boolean_1 = pd.DataFrame()
outlier_boolean_2 = pd.DataFrame()

for sal_class in master_sample['sal_class'].unique():
    subset = master_sample.query('sal_class == @sal_class').set_index(['financial_statement_id'])[RATIOS]

    outlier_boolean_1 = pd.concat([
        outlier_boolean_1,
        flag_outliers_mahalanobis(subset[RATIOS]).to_frame(name = 'is_mahalanobis_outlier').reset_index()
    ], axis=0)
            
    outlier_boolean_2 = pd.concat([
        outlier_boolean_2,
        detect_outliers_isolation_forest(subset[RATIOS]).to_frame(name = 'is_isolation_forest_outlier').reset_index()
    ], axis=0)



display(outlier_boolean_1['is_mahalanobis_outlier'].value_counts())
display(outlier_boolean_2['is_isolation_forest_outlier'].value_counts())

c:\Users\matus.rohal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
c:\Users\matus.rohal\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


False    8887
True      101
Name: is_mahalanobis_outlier, dtype: int64

False    8807
True      181
Name: is_isolation_forest_outlier, dtype: int64

In [27]:

display(outlier_boolean_1['is_mahalanobis_outlier'].value_counts(normalize = True))
display(outlier_boolean_2['is_isolation_forest_outlier'].value_counts(normalize = True))

False    0.988763
True     0.011237
Name: is_mahalanobis_outlier, dtype: float64

False    0.979862
True     0.020138
Name: is_isolation_forest_outlier, dtype: float64

In [28]:
for col in ['is_mahalanobis_outlier', 'is_isolation_forest_outlier']:
    # if col in master_sample.columns: print('drop')
    if col in master_sample.columns: master_sample.drop(columns=[col], inplace=True)

In [29]:
display(master_sample.shape)

master_sample = master_sample.merge(outlier_boolean_1, on = ['financial_statement_id'], how = 'left')
master_sample = master_sample.merge(outlier_boolean_2, on = ['financial_statement_id'], how = 'left')

display(master_sample.shape)
display(master_sample)

(8988, 99)

(8988, 101)

,entity_id,year,month,financial_report_id,financial_report_template_id,CA,CASH,CL,EQ,NCL,TA,TL,CC,INV,CF_NETTO,CF_SELFFIN,REV,VA,OE,EAT,EBIT,SAL,COST,INT,LABOR,L1,L2,L3,NCL_CF,CASH_TA,INV_COGS,CC_SAL,TL_COST,CL_COST,SAL_TA,EQ_TA,EQ_TL,CL_CC,LTC_NCA,TZ,CF_TL,CL_TA,NCL_TA,EBT_REV,ROS,EAT_REV,ROA,ROE,CF_TA,CF_SAL,...,LABOR_REV,financial_statement_id,financial_report_order_number,entity_ico,period_length,consolidated,approved_date,type,establishment_date,cancellation_date,sknace_code,entity_state,legal_form_code,org_size_code,ownership_category_code,entity_consolidated,sknace_division_name,sknace_division,sknace_subcategory,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,DPHZ_vat_registration_cancelled_3y_off,RU_event,RU_event_1y_off,RU_event_2y_off,RU_event_3y_off,cancellation_year,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_cancelled_3y_off,sal_class,establishment_year,sknace_division_normalized,entity_name,parsed_state,RUZ_indirect_criteria,RUZ_indirect_criteria_1y_off,RUZ_indirect_criteria_2y_off,RUZ_indirect_criteria_3y_off,combined_issue_criteria_1y_off,combined_issue_criteria_2y_off,combined_issue_criteria_3y_off,RUZ_liquidation_3y_off,RUZ_liquidation_2y_off,RUZ_liquidation_1y_off,plausible_sample,is_mahalanobis_outlier,is_isolation_forest_outlier
0,27971.0,2018.0,1.0,6791492,699.0,824851.0,137120.0,564418.0,17376.0,46374.0,875393.0,846466.0,673298.0,11072.0,13087.0,9028.0,3031970.0,260410.0,24567.0,2651.0,6710.0,3031963.0,3007396.0,8218.0,370870.0,0.242941,1.435847,1.455464,3.543516,0.156638,NaN,0.222067,0.281461,0.187677,3.463545,0.019849,0.020115,0.838289,5.924261,1.908459,0.015461,0.644760,0.052975,0.002213,0.000874,0.000874,0.003028,0.152567,0.014950,0.004316,...,0.122320,4101516.0,1.0,652423.0,12.0,NaN,2019-06-28,Riadna,1991-06-25,None,49410.0,NaN,112.0,2.0,2.0,False,Pozemná doprava a doprava potrubím,H,49.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,0,0,0,0,2. Malý,1991,H,KUHO - mix. spol. s r.o.,ok,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0,0,1.0,False,False
1,60314.0,2016.0,1.0,5545966,699.0,58815.0,15639.0,56837.0,127982.0,76345.0,2248496.0,1745181.0,23576.0,18552.0,2741491.0,2731211.0,3322652.0,199899.0,-472509.0,2073411.0,2079171.0,692195.0,1164704.0,1958.0,224570.0,0.275155,0.689955,1.016363,0.027848,0.006955,2.734670,0.034060,1.498390,0.048800,0.307848,0.056919,0.068324,2.410799,0.829493,0.615510,1.570892,0.025278,0.033954,0.625756,2.995415,0.624023,0.922132,16.200802,1.219255,3.960576,...,0.067588,3090140.0,1.0,36589993.0,12.0,NaN,None,Riadna,2005-02-22,2021-06-24,11050.0,NaN,112.0,0.0,8.0,False,Výroba nápojov,C,11.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2021.0,0,0,0,0,2. Malý,2005,C,Pivovar Kaltenecker s.r.o. v konkurze,konkurz,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0,0,0,1.0,False,False
2,71752.0,2015.0,1.0,5058691,699.0,912862.0,2213.0,48840.0,2399.0,18406.0,948368.0,945969.0,910649.0,0.0,-35846.0,-38726.0,2179262.0,-27686.0,-68089.0,-57122.0,-54242.0,2134583.0,2202672.0,67110.0,12114.0,0.045311,18.690868,18.690868,-0.513474,0.002333,0.000000,0.426617,0.429464,0.022173,2.250796,0.002530,0.002530,0.053632,25.334535,-0.923590,-0.037893,0.051499,0.019408,-0.024890,-0.026760,-0.026212,-0.060232,-23.810754,-0.037798,-0.016793,...,0.005559,2703097.0,1.0,36584649.0,12.0,NaN,2016-03-10,Riadna,2004-10-08,None,46720.0,NaN,112.0,0.0,8.0,False,"Veľkoobchod, okrem motorových vozidiel a motoc...",G,46.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,0,0,0,0,2. Malý,2004,G,FeCom s.r.o. v konkurze,konkurz,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0,0,1.0,False,False
3,72912.0,2017.0,1.0,6351388,699.0,1481047.0,1630.0,1326396.0,-718007.0,549510.0,1521883.0,2239890.0,1417869.0,61548.0,47298.0,44418.0,4512573.0,802511.0,52068.0,10312.0,13192.0,4512342.0,4460274.0,32121.0,692228.0,0.001229,1.070192,1.116595,11.618039,0.001071,NaN,0.314220,0.502187,0.297380,2.964973,-0.471789,-0.471789,0.935486,4.787124,16.043871,0.021116,0.871549,0.361072,0.002923,0.002285,0.002285,0.006776,-0.014362,0.031079,0.010482,...,

In [30]:
display(get_sample_sizes(master_sample, 'is_mahalanobis_outlier', 'combined_issue_criteria_3y_off'))
display(get_sample_sizes(master_sample, 'is_isolation_forest_outlier', 'combined_issue_criteria_3y_off'))

count     
combined_issue_criteria_3y_off   0.0  1.0
is_mahalanobis_outlier                   
False                           8048  839
True                              68   33

count     
combined_issue_criteria_3y_off   0.0  1.0
is_isolation_forest_outlier              
False                           7973  834
True                             143   38

In [31]:
master_sample = master_sample.query('sknace_division_normalized != "non-relevant"')
testing_sample = master_sample.sample(frac=0.2, random_state=17)
testing_sample_ids = set(testing_sample['entity_id'])

In [32]:
training_sample = master_sample.query('entity_id not in @testing_sample_ids')

In [33]:
for dim in ['RU_event_3y_off', 'RUZ_cancelled_3y_off', 'RUZ_indirect_criteria_3y_off', 'combined_issue_criteria_3y_off']:
    print(dim)
    display(
        pd.concat([
            master_sample[dim].value_counts(normalize=True).to_frame('dataset'),
            training_sample[dim].value_counts(normalize=True).to_frame('training_sample'),
            testing_sample[dim].value_counts(normalize=True).to_frame('testing_sample')
        ], axis = 1)
    )

RU_event_3y_off


,dataset,training_sample,testing_sample
0.0,0.974402,0.973428,0.978297
1.0,0.025598,0.026572,0.021703


RUZ_cancelled_3y_off


,dataset,training_sample,testing_sample
0,0.988759,0.98887,0.988314
1,0.011241,0.01113,0.011686


RUZ_indirect_criteria_3y_off


,dataset,training_sample,testing_sample
0.0,0.92621,0.926544,0.924875
1.0,0.07379,0.073456,0.075125


combined_issue_criteria_3y_off


,dataset,training_sample,testing_sample
0.0,0.902949,0.902615,0.904285
1.0,0.097051,0.097385,0.095715


In [34]:
master_sample.to_csv('../../DATA/FINAL/super_sample.csv', index=False)
training_sample.to_csv('../../DATA/FINAL/training_sample.csv', index=False)
testing_sample.to_csv('../../DATA/FINAL/testing_sample.csv', index=False)

In [35]:
for dim in DIMENSIONS:
    print(dim)
    display(
        pd.concat([
            master_sample[dim].value_counts(normalize=True).to_frame('dataset'),
            training_sample[dim].value_counts(normalize=True).to_frame('training_sample'),
            testing_sample[dim].value_counts(normalize=True).to_frame('testing_sample')
        ], axis = 1)
    )

year


,dataset,training_sample,testing_sample
2015.0,0.256984,0.253200,0.272120
2016.0,0.233278,0.231775,0.239288
2017.0,0.235726,0.236505,0.232610
2018.0,0.274012,0.278520,0.255982


period_length


,dataset,training_sample,testing_sample
12.0,1.0,1.0,1.0


type


...

,dataset,training_sample,testing_sample
G,0.322204,0.322760,0.319978
C,0.205120,0.206873,0.198108
F,0.092042,0.090568,0.097941
M,0.079354,0.079855,0.077351
H,0.067112,0.068865,0.060100
N,0.050751,0.052309,0.044519
L,0.049193,0.048831,0.050640
J,0.034613,0.033250,0.040067
A,0.030161,0.029772,0.031720
I,0.013578,0.013216,0.015025


sknace_division_normalized


,dataset,training_sample,testing_sample
G,0.322204,0.322760,0.319978
other,0.234168,0.231080,0.246522
C,0.205120,0.206873,0.198108
F,0.092042,0.090568,0.097941
M,0.079354,0.079855,0.077351
H,0.067112,0.068865,0.060100


sal_class


,dataset,training_sample,testing_sample
2. Malý,0.815915,0.818587,0.805231
3. Stredný,0.184085,0.181413,0.194769


In [36]:
for dim in DIMENSIONS:
    print(dim)
    display(
        pd.concat([
            master_sample[dim].value_counts(normalize=True).to_frame('dataset'),
            training_sample[dim].value_counts(normalize=True).to_frame('training_sample'),
            testing_sample[dim].value_counts(normalize=True).to_frame('testing_sample')
        ], axis = 1)
    )

year


,dataset,training_sample,testing_sample
2015.0,0.256984,0.253200,0.272120
2016.0,0.233278,0.231775,0.239288
2017.0,0.235726,0.236505,0.232610
2018.0,0.274012,0.278520,0.255982


period_length


,dataset,training_sample,testing_sample
12.0,1.0,1.0,1.0


type


...

,dataset,training_sample,testing_sample
G,0.322204,0.322760,0.319978
C,0.205120,0.206873,0.198108
F,0.092042,0.090568,0.097941
M,0.079354,0.079855,0.077351
H,0.067112,0.068865,0.060100
N,0.050751,0.052309,0.044519
L,0.049193,0.048831,0.050640
J,0.034613,0.033250,0.040067
A,0.030161,0.029772,0.031720
I,0.013578,0.013216,0.015025


sknace_division_normalized


,dataset,training_sample,testing_sample
G,0.322204,0.322760,0.319978
other,0.234168,0.231080,0.246522
C,0.205120,0.206873,0.198108
F,0.092042,0.090568,0.097941
M,0.079354,0.079855,0.077351
H,0.067112,0.068865,0.060100


sal_class


,dataset,training_sample,testing_sample
2. Malý,0.815915,0.818587,0.805231
3. Stredný,0.184085,0.181413,0.194769
